<a href="https://colab.research.google.com/github/Khozinkosi/Sec2sec-model/blob/main/Assignment3Comp703.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Wandile Ngobese (222056013)
*   Linda Sekhoasha (222004139)
*   Khonzinkosi Mkhize (219005273)
*   Samukelo Mkhize (220009930)


In [2]:
!pip install nltk

In [4]:
!pip install tqdm

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Bidirectional
import matplotlib.pyplot as plt
import os
import zipfile
import urllib.request
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')  # Required for tokenization
from tqdm import tqdm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Load and Process dataset**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset_path = '/content/drive/My Drive/fra-eng/fra.txt'

In [5]:
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = file.read().strip().split('\n')

In [6]:
input_texts = []
target_texts = []

for line in lines:
    parts = line.strip().split('\t')
    eng, fra = parts[0], parts[1]
    input_texts.append(eng)
    target_texts.append('\t' + fra + '\n')  # Add start '\t' and end '\n' tokens

In [7]:
for i in range(-5, 0):
    print(f"{i+len(input_texts)}: {input_texts[i]}")


237833: Death is something that we're often discouraged to talk about or even think about, but I've realized that preparing for death is one of the most empowering things you can do. Thinking about death clarifies your life.
237834: Since there are usually multiple websites on any given topic, I usually just click the back button when I arrive on any webpage that has pop-up advertising. I just go to the next page found by Google and hope for something less irritating.
237835: If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.
237836: It may be impossible to get a completely error-free corpus due to the nature of this kind of collaborative effort. However, if we encourage members to contribute sentences in their own languages rather than experiment in languages they are learning

In [8]:
for i in range(5):
    print(f"English: {input_texts[i]}")
    print(f"French : {target_texts[i]}")
    print('---')

English: Go.
French : 	Va !

---
English: Go.
French : 	Marche.

---
English: Go.
French : 	En route !

---
English: Go.
French : 	Bouge !

---
English: Hi.
French : 	Salut !

---


In [9]:
input_characters = sorted(set(char for text in input_texts for char in text))
target_characters = sorted(set(char for text in target_texts for char in text))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max(len(text) for text in input_texts)
max_decoder_seq_length = max(len(text) for text in target_texts)


**Then create mappings:**

In [10]:
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}


** Create integer-encoded sequences**

In [11]:
input_token_index = {char: i + 1 for i, char in enumerate(input_characters)}  # reserve 0 for padding
target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}

encoder_input_seq = []
decoder_input_seq = []
decoder_target_seq = []

for i in range(len(input_texts)):
    input_seq = [input_token_index[char] for char in input_texts[i]]
    target_seq = [target_token_index[char] for char in target_texts[i]]

    decoder_input = target_seq[:-1]
    decoder_target = target_seq[1:]

    encoder_input_seq.append(input_seq)
    decoder_input_seq.append(decoder_input)
    decoder_target_seq.append(decoder_target)


**Create one-hot encoded 3D arrays for encoder input, decoder input, and decoder target:**

In [12]:
'''import numpy as np

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        #encoder_input_data[i, t, input_token_index[char]] = 1.0
        num_encoder_tokens = len(input_token_index)  # Should be correct

    for t, char in enumerate(target_text):
        target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}
        num_decoder_tokens = len(target_token_index) + 1  # +1 is CRUCIAL!



        if t > 0:
            # decoder target data is ahead by one timestep
            target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}  # Start from 1'''


import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# -----------------------------
# Step 1: Build token dictionaries
# -----------------------------
input_token_index = {char: i + 1 for i, char in enumerate(input_characters)}  # start from 1
target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}  # start from 1

num_encoder_tokens = len(input_token_index) + 1  # +1 for padding (index 0)
num_decoder_tokens = len(target_token_index) + 1

# -----------------------------
# Step 2: Convert characters to integer sequences
# -----------------------------
encoder_input_sequences = []
decoder_input_sequences = []
decoder_target_sequences = []

for i in range(len(input_texts)):
    input_seq = [input_token_index[char] for char in input_texts[i]]
    target_seq = [target_token_index[char] for char in target_texts[i]]

    encoder_input_sequences.append(input_seq)
    decoder_input_sequences.append(target_seq[:-1])  # decoder input excludes last char
    decoder_target_sequences.append(target_seq[1:])  # decoder target excludes first char

# -----------------------------
# Step 3: Pad sequences
# -----------------------------
encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_sequences, maxlen=max_decoder_seq_length, padding='post')

# -----------------------------
# Step 4: Reshape decoder target for training
# -----------------------------
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# -----------------------------
# Step 5: Define the model with Embeddings
# -----------------------------
latent_dim = 128  # Feel free to adjust based on your system

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=64)(decoder_inputs)
decoder_lstm, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Combined Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 64)  │      5,824 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 64)  │      7,424 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │     98,816 │ embedding[0][0]   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     98,816 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 116) │     14,964 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 225,844 (882.20 KB)

 Trainable params: 225,844 (882.20 KB)

 Non-trainable params: 0 (0.00 B)

Bidirectional LSTM Encoder + LSTM Decoder

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional

# Set latent dimensionality
latent_dim = 128  # You can adjust this based on memory

# Define encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_inputs)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None, 91)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 256),     │    225,280 │ input_layer_2[0]… │
│ (Bidirectional)     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None, 116) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    381,952 │ input_layer_3[0]… │
│                     │ 256), (None,      │            │ concatenate[0][0… │
│                     │ 256), (None,      │            │ concatenate_1[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 116) │     29,812 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 637,044 (2.43 MB)

 Trainable params: 637,044 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

**imit Number of Samples **

In [14]:
MAX_SAMPLES = 5000  # or start with 20000
filtered_input = []
filtered_target = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 2:
        eng, fra = parts
        filtered_input.append(eng)
        filtered_target.append('\t' + fra + '\n')
    if len(filtered_input) >= MAX_SAMPLES:
        break

input_texts = filtered_input
target_texts = filtered_target


In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=64)(decoder_inputs)
decoder_lstm, _, _ = LSTM(128, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Train the Model

In [16]:
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)


Epoch 1/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 109s 35ms/step - accuracy: 0.9302 - loss: 0.2755 - val_accuracy: 0.9029 - val_loss: 0.3297
Epoch 2/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 140s 35ms/step - accuracy: 0.9522 - loss: 0.1625 - val_accuracy: 0.9145 - val_loss: 0.2912
Epoch 3/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 142s 36ms/step - accuracy: 0.9574 - loss: 0.1439 - val_accuracy: 0.9193 - val_loss: 0.2724
Epoch 4/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 105s 35ms/step - accuracy: 0.9600 - loss: 0.1343 - val_accuracy: 0.9222 - val_loss: 0.2608
Epoch 5/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 142s 35ms/step - accuracy: 0.9618 - loss: 0.1279 - val_accuracy: 0.9249 - val_loss: 0.2513
Epoch 6/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 142s 35ms/step - accuracy: 0.9630 - loss: 0.1231 - val_accuracy: 0.9265 - val_loss: 0.2453
Epoch 7/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 142s 35ms/step - accuracy: 0.9641 - loss: 0.1194 - val_accuracy: 0.9281 - val_loss: 0.2390
Epoch 8/10
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 115s 39ms/step - accuracy: 

**Bidirectional LSTM Encoder + LSTM Decoder**

**Create a function that evaluates your model's translations using the BLEU score.**

In [17]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def decode_sequence(input_seq, encoder_model, decoder_model, max_decoder_seq_length, target_token_index, reverse_target_char_index):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with just the start character.
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    decoded_sentence = ''
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        if sampled_char == '\n':
            break
        decoded_sentence += sampled_char

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

# Placeholder: define encoder_model and decoder_model later (after training)


**Set up inference models (separate encoder and decoder) for testing.**

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Decoder states input
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder input
decoder_inputs_inf = Input(shape=(None,))

# Reuse decoder embedding layer (correct name: embedding_3)
decoder_embedding_layer = model.get_layer(name='embedding_3')
decoder_embedding_inf = decoder_embedding_layer(decoder_inputs_inf)

# Reuse decoder LSTM layer (correct name: lstm_5)
decoder_lstm_layer = model.get_layer(name='lstm_5')
decoder_outputs, state_h, state_c = decoder_lstm_layer(
    decoder_embedding_inf, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]

# Reuse final Dense layer (correct name: dense_2)
decoder_dense_layer = model.get_layer(name='dense_2')
decoder_outputs = decoder_dense_layer(decoder_outputs)

# Final decoder model for inference
decoder_model = Model(
    [decoder_inputs_inf] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

**This function uses the inference models to translate an English sentence (as a character sequence) into French.**

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with just the start character '\t'
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    # Initialize decoded sentence
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char

        # Exit if end token or max length reached
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


**Create an evaluation wrapper with BLEU score**

In [24]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

def evaluate_bleu(n=100):
    total_bleu = 0.0
    for i in range(n):
        input_seq = encoder_input_data[i:i+1]  # One sample
        decoded_sentence = decode_sequence(input_seq)

        # Remove start/end tokens
        reference = [list(target_texts[i][1:-1])]
        candidate = list(decoded_sentence.strip())

        bleu = sentence_bleu(reference, candidate, smoothing_function=smoothie)
        total_bleu += bleu

        print(f'Input: {input_texts[i]}')
        print(f'Target: {target_texts[i]}')
        print(f'Predicted: {decoded_sentence.strip()}')
        print(f'BLEU: {bleu:.4f}')
        print('---')

    avg_bleu = total_bleu / n
    print(f'\nAverage BLEU score over {n} samples: {avg_bleu:.4f}')


**Run the evaluation**

In [29]:
def evaluate_bleu(n=50):
    total_bleu = 0.0
    num_samples = min(n, len(input_texts))  # Make sure not to exceed available data

    if num_samples == 0:
        print("No samples available for evaluation.")
        return

    for i in range(num_samples):
        input_seq = encoder_input_data[i:i+1]
        decoded_sentence = decode_sequence(input_seq)

        # Reference: remove start '\t' and end '\n'
        reference = [list(target_texts[i][1:-1])]
        candidate = list(decoded_sentence.strip())

        bleu = sentence_bleu(reference, candidate, smoothing_function=smoothie)
        total_bleu += bleu

        print(f'Input: {input_texts[i]}')
        print(f'Target: {target_texts[i]}')
        print(f'Predicted: {decoded_sentence.strip()}')
        print(f'BLEU: {bleu:.4f}')
        print('---')

    avg_bleu = total_bleu / num_samples
    print(f'\nAverage BLEU score over {num_samples} samples: {avg_bleu:.4f}')

evaluate_bleu(n=50)


No samples available for evaluation.


In [32]:
print("Number of input_texts:", len(input_texts))
print("Number of target_texts:", len(target_texts))
print("encoder_input_data shape:", encoder_input_data.shape)
print("decoder_input_data shape:", decoder_input_data.shape)
print("decoder_target_data shape:", decoder_target_data.shape)


Number of input_texts: 237838
Number of target_texts: 237838
encoder_input_data shape: (237838, 292)
decoder_input_data shape: (237838, 351)
decoder_target_data shape: (237838, 351, 1)


**Restore original texts from encoder input**

In [31]:
# Reconstruct input_texts from encoder_input_data
reversed_input_token_index = {i: char for char, i in input_token_index.items()}
reversed_target_token_index = {i: char for char, i in target_token_index.items()}

def decode_sequence_from_input(input_seq):
    return ''.join(reversed_input_token_index.get(i, '') for i in input_seq if i != 0)

def decode_sequence_from_target(target_seq):
    return ''.join(reversed_target_token_index.get(i, '') for i in target_seq if i != 0)

# Rebuild input_texts and target_texts for BLEU
input_texts = [decode_sequence_from_input(seq) for seq in encoder_input_data]
target_texts = ['\t' + decode_sequence_from_target(seq) + '\n' for seq in decoder_input_data]


In [33]:
from nltk.translate.bleu_score import sentence_bleu

def decode_sequence(input_seq):
    # Encode the input as states
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']  # Start token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

def evaluate_bleu(n=50):
    total_bleu = 0
    num_samples = min(n, len(input_texts))

    for i in range(num_samples):
        input_seq = encoder_input_data[i:i+1]
        decoded_sentence = decode_sequence(input_seq)

        reference = [list(target_texts[i][1:-1])]  # Remove \t and \n
        candidate = list(decoded_sentence.strip())

        bleu_score = sentence_bleu(reference, candidate, weights=(0.5, 0.5))
        total_bleu += bleu_score

        print(f'Input    : {input_texts[i]}')
        print(f'Target   : {target_texts[i][1:-1]}')
        print(f'Predicted: {decoded_sentence.strip()}')
        print(f'BLEU     : {bleu_score:.4f}')
        print('---')

    avg_bleu = total_bleu / num_samples
    print(f'\nAverage BLEU score over {num_samples} samples: {avg_bleu:.4f}')

# Run the evaluation on N samples
evaluate_bleu(n=50)


Input    : Go.
Target   : 	Va !
Predicted: Kf!of!tvjt!qbt!ef!mb!qpsuf!ef!mb!qpsuf/ / / / / / «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A
BLEU     : 0.0040
---


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Input    : Go.
Target   : 	Marche.
Predicted: Kf!of!tvjt!qbt!ef!mb!qpsuf!ef!mb!qpsuf/ / / / / / «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A
BLEU     : 0.0000
---
Input    : Go.
Target   : 	En route !
Predicted: Kf!of!tvjt!qbt!ef!mb!qpsuf!ef!mb!qpsuf/ / / / / / «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A «A
BLEU     : 0.0075
---
Input    : Go.
Target   : 	Bouge !
Predicted: Kf!of!tvjt!qbt!ef!mb!qpsuf!ef!mb!qpsuf/ / / / / / «A !ftu!mf!gbjsf!A «A !A !ftu.df!A «A «A «A !ftu!mf!gbjsf!A

KeyboardInterrupt: 